Cell 1 — Setup: imports, paths, and output folders

Purpose: load libraries and define the LIGHT dataset + output files we will generate.

In [4]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image

tqdm.pandas()

CSV_LIGHT = "data/laion_big_light_tau1_0.2989.csv"

OUT_DIR = Path("data/week4_dedup_light")
OUT_DIR.mkdir(parents=True, exist_ok=True)

PHASH_CACHE = OUT_DIR / "light_with_phash_cache.csv"   # progressive save
DEDUP_OUT = OUT_DIR / "light_phash_dedup.csv"
SUMMARY_OUT = OUT_DIR / "week4_light_dedup_summary.json"

print("Input:", CSV_LIGHT)
print("Output dir:", OUT_DIR)


Input: data/laion_big_light_tau1_0.2989.csv
Output dir: data\week4_dedup_light


Cell 2 — Load LIGHT dataset and inspect columns

Purpose: confirm dataset size and check which metadata columns are available.

In [5]:
df = pd.read_csv(CSV_LIGHT)
print("Rows:", len(df))
print("Columns:", df.columns.tolist())
df.head(3)


Rows: 107251
Columns: ['image_path', 'caption', 'width', 'height', 'similarity', 'punsafe', 'pwatermark', 'aesthetic_score', 'caption_len_words', 'exists', 'keep_quality', 'clipscore']


,image_path,caption,width,height,similarity,punsafe,pwatermark,aesthetic_score,caption_len_words,exists,keep_quality,clipscore
0,src/data/data/laion_aesthetic_subset_big\00000...,Photo pour Japanese pagoda and old house in Ky...,450.0,297.0,0.345947,0.000541,0.034099,6.526204,16,True,True,0.335841
1,src/data/data/laion_aesthetic_subset_big\00000...,San Pedro: One Of Mother Nature's Most Powerfu...,467.0,369.0,0.308873,0.000612,0.102705,6.749783,14,True,True,0.300125
2,src/data/data/laion_aesthetic_subset_big\00000...,"YZV200025S © Stocktrek Images, Inc. 360 panora...",650.0,308.0,0.332100,0.000013,0.057178,6.529371,15,True,True,0.342683


Cell 3 — Keep only rows with existing images (mandatory)

Purpose: remove rows whose image files are missing to avoid errors during hashing.

In [6]:
df["exists"] = df["image_path"].astype(str).apply(os.path.exists)
print("Existing images:", int(df["exists"].sum()), "/", len(df))

df = df[df["exists"]].copy()
df.reset_index(drop=True, inplace=True)
print("After dropping missing images:", len(df))


Existing images: 107251 / 107251
After dropping missing images: 107251


Cell 4 — Diversity snapshot BEFORE deduplication

Purpose: compute baseline statistics to compare diversity before vs after deduplication.

In [7]:
def caption_len_words(x):
    if not isinstance(x, str):
        return 0
    return len(x.split())

df["caption_len_words"] = df["caption"].apply(caption_len_words)

stats_before = {
    "rows": int(len(df)),
    "caption_len_mean": float(df["caption_len_words"].mean()),
    "caption_len_median": float(df["caption_len_words"].median()),
    "width_mean": float(df["width"].mean()) if "width" in df.columns else None,
    "height_mean": float(df["height"].mean()) if "height" in df.columns else None,
    "aesthetic_mean": float(df["aesthetic_score"].mean()) if "aesthetic_score" in df.columns else None,
    "clipscore_mean": float(df["clipscore"].mean()) if "clipscore" in df.columns else None,
}
stats_before


{'rows': 107251,
 'caption_len_mean': 11.50586008522065,
 'caption_len_median': 9.0,
 'width_mean': 640.2439884010405,
 'height_mean': 555.3142441562317,
 'aesthetic_mean': 6.638642123141852,
 'clipscore_mean': 0.3257998502730627}

Cell 5 — Exact deduplication by image_path (fast)

Purpose: remove exact repeated entries that point to the same local file path.

In [8]:
dup_paths = int(df.duplicated("image_path").sum())
print("Exact duplicates by image_path:", dup_paths)

df_exact = df.drop_duplicates("image_path").copy()
df_exact.reset_index(drop=True, inplace=True)

print("Before:", len(df), "After exact path dedup:", len(df_exact))


Exact duplicates by image_path: 0
Before: 107251 After exact path dedup: 107251


Cell 6 — Install ImageHash (only if not installed)

Purpose: ensure the library for perceptual hashing (pHash) is available.

In [9]:
pip install imagehash


   ---------------------------------------- 0.0/4.2 MB ? eta -:--:--
   ---------------------- ----------------- 2.4/4.2 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 4.2/4.2 MB 11.0 MB/s  0:00:00
   ---------------------------------------- 0.0/41.3 MB ? eta -:--:--
   -- ------------------------------------- 2.4/41.3 MB 12.2 MB/s eta 0:00:04
   ---- ----------------------------------- 4.7/41.3 MB 11.9 MB/s eta 0:00:04
   ------- -------------------------------- 7.3/41.3 MB 11.9 MB/s eta 0:00:03
   --------- ------------------------------ 9.7/41.3 MB 11.8 MB/s eta 0:00:03
   ----------- ---------------------------- 12.1/41.3 MB 11.8 MB/s eta 0:00:03
   ------------- -------------------------- 14.4/41.3 MB 11.8 MB/s eta 0:00:03
   ---------------- ----------------------- 17.0/41.3 MB 11.8 MB/s eta 0:00:03
   ------------------ --------------------- 19.4/41.3 MB 11.8 MB/s eta 0:00:02
   --------------------- ------------------ 21.8/41.3 MB 11.8 MB/s eta 0:00:02
   

Cell 7 — Define pHash function (robust image loading)

Purpose: compute a perceptual hash that detects visually similar images (near-duplicates).

In [10]:
import imagehash

def compute_phash(path: str) -> str:
    img = Image.open(path).convert("RGB")
    return str(imagehash.phash(img))  # returns a compact hash string


Cell 8 — Resume if cache exists (avoid recomputing pHash)

Purpose: if the notebook stops, reload previous hashes and continue from remaining images.

In [11]:
if PHASH_CACHE.exists():
    print("✅ Found cache:", PHASH_CACHE)
    df_hash = pd.read_csv(PHASH_CACHE)
    # Ensure correct columns exist
    if "phash" not in df_hash.columns:
        df_hash["phash"] = np.nan
else:
    print("ℹ️ No cache found. Creating a new one.")
    df_hash = df_exact.copy()
    df_hash["phash"] = np.nan

print("Rows in working dataframe:", len(df_hash))
print("Already hashed:", int(df_hash["phash"].notna().sum()))
print("Remaining:", int(df_hash["phash"].isna().sum()))



ℹ️ No cache found. Creating a new one.
Rows in working dataframe: 107251
Already hashed: 0
Remaining: 107251


Cell 9 — Compute pHash for ALL 107k (progressive checkpointing)

Purpose: compute pHash for every image and save progress regularly to avoid losing work.

In [12]:
SAVE_EVERY = 1000  # save every N images

remaining_idx = df_hash.index[df_hash["phash"].isna()].to_list()
print("To hash:", len(remaining_idx))

for k, i in enumerate(tqdm(remaining_idx, desc="🔐 Computing pHash (LIGHT 107k)")):
    try:
        df_hash.at[i, "phash"] = compute_phash(df_hash.at[i, "image_path"])
    except Exception:
        df_hash.at[i, "phash"] = np.nan  # keep as NaN if image is corrupted

    if (k + 1) % SAVE_EVERY == 0:
        df_hash.to_csv(PHASH_CACHE, index=False)
        print(f"💾 Saved checkpoint: {PHASH_CACHE} | hashed: {int(df_hash['phash'].notna().sum())}/{len(df_hash)}")

# final save
df_hash.to_csv(PHASH_CACHE, index=False)
print("✅ Final pHash cache saved:", PHASH_CACHE)



To hash: 107251


🔐 Computing pHash (LIGHT 107k):   0%|                                                 | 0/107251 [00:00<?, ?it/s]C:\Users\kamen\AppData\Local\Temp\ipykernel_27176\1573748848.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'c9c9ab4a68d0d8f9' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_hash.at[i, "phash"] = compute_phash(df_hash.at[i, "image_path"])
🔐 Computing pHash (LIGHT 107k):   1%|▎                                     | 1022/107251 [00:08<46:37, 37.98it/s]

💾 Saved checkpoint: data\week4_dedup_light\light_with_phash_cache.csv | hashed: 1000/107251


🔐 Computing pHash (LIGHT 107k):   2%|▋                                     | 2016/107251 [00:16<43:59, 39.87it/s]

💾 Saved checkpoint: data\week4_dedup_light\light_with_phash_cache.csv | hashed: 2000/107251


...

💾 Saved checkpoint: data\week4_dedup_light\light_with_phash_cache.csv | hashed: 106000/107251


🔐 Computing pHash (LIGHT 107k): 100%|███████████████████████████████████▉| 107017/107251 [15:11<00:05, 46.80it/s]

💾 Saved checkpoint: data\week4_dedup_light\light_with_phash_cache.csv | hashed: 107000/107251


🔐 Computing pHash (LIGHT 107k): 100%|███████████████████████████████████| 107251/107251 [15:12<00:00, 117.52it/s]


✅ Final pHash cache saved: data\week4_dedup_light\light_with_phash_cache.csv


Cell 10 — Analyze near-duplicate structure using pHash

Purpose: measure how many duplicates exist and identify the most repeated visual groups.

In [13]:
valid = df_hash[df_hash["phash"].notna()].copy()
unique_phash = valid["phash"].nunique()

print("Valid hashes:", len(valid), "/", len(df_hash))
print("Unique pHash:", unique_phash, "/", len(valid))
print("Approx near-duplicate rate:", round(1 - unique_phash/len(valid), 4))

groups = valid.groupby("phash").size().sort_values(ascending=False)
print("Top duplicate groups:")
display(groups.head(10))


Valid hashes: 107251 / 107251
Unique pHash: 81061 / 107251
Approx near-duplicate rate: 0.2442
Top duplicate groups:


phash
88890b7d497c75f8    190
91efe9b122d29c4a    188
c9c81f26234764f7    111
95c4e26a5b54e476     54
eca413f6839bcc86     45
d118d5ae71e215ec     41
83c75e277cc076c8     40
c29d1de319302f5e     39
ea7ed512d441dc94     33
e0464fdd6e0a9966     32
dtype: int64

Cell 11 — pHash deduplication (keep one image per group)

Purpose: build a deduplicated dataset by keeping a single representative per pHash group.

In [14]:
df_phash_dedup = valid.drop_duplicates("phash").copy()
df_phash_dedup.reset_index(drop=True, inplace=True)

print("Before (valid hashed):", len(valid))
print("After pHash dedup:", len(df_phash_dedup))



Before (valid hashed): 107251
After pHash dedup: 81061


Cell 12 — Diversity snapshot AFTER deduplication

Purpose: compare diversity statistics before vs after deduplication for the report.

In [15]:
df_phash_dedup["caption_len_words"] = df_phash_dedup["caption"].apply(caption_len_words)

stats_after = {
    "rows": int(len(df_phash_dedup)),
    "caption_len_mean": float(df_phash_dedup["caption_len_words"].mean()),
    "caption_len_median": float(df_phash_dedup["caption_len_words"].median()),
    "width_mean": float(df_phash_dedup["width"].mean()) if "width" in df_phash_dedup.columns else None,
    "height_mean": float(df_phash_dedup["height"].mean()) if "height" in df_phash_dedup.columns else None,
    "aesthetic_mean": float(df_phash_dedup["aesthetic_score"].mean()) if "aesthetic_score" in df_phash_dedup.columns else None,
    "clipscore_mean": float(df_phash_dedup["clipscore"].mean()) if "clipscore" in df_phash_dedup.columns else None,
}
stats_after


{'rows': 81061,
 'caption_len_mean': 11.416291434845363,
 'caption_len_median': 9.0,
 'width_mean': 653.9466697918851,
 'height_mean': 562.16467845203,
 'aesthetic_mean': 6.634265351825774,
 'clipscore_mean': 0.3249492619067096}

Cell 13 — Save deduplicated dataset + JSON summary

Purpose: produce clean deliverables for your partner and for reporting.

In [16]:
df_phash_dedup.to_csv(DEDUP_OUT, index=False)
print("✅ Saved deduplicated CSV:", DEDUP_OUT)

import json

summary = {
    "input_csv": CSV_LIGHT,
    "rows_input_existing": int(len(df)),
    "rows_after_exact_path_dedup": int(len(df_exact)),
    "rows_hashed_valid": int(len(valid)),
    "rows_after_phash_dedup": int(len(df_phash_dedup)),
    "stats_before": stats_before,
    "stats_after": stats_after,
}

SUMMARY_OUT.write_text(json.dumps(summary, indent=2), encoding="utf-8")
print("✅ Saved summary JSON:", SUMMARY_OUT)


✅ Saved deduplicated CSV: data\week4_dedup_light\light_phash_dedup.csv
✅ Saved summary JSON: data\week4_dedup_light\week4_light_dedup_summary.json


Cell 14 — (Optional) Quick qualitative inspection of duplicates

Purpose: inspect a duplicate group to confirm the deduplication behavior visually.

In [17]:
# Pick a hash with many duplicates (if any)
top_hash = groups.index[0]
dup_examples = valid[valid["phash"] == top_hash].head(5)[["image_path", "caption", "clipscore", "aesthetic_score"]]
dup_examples


,image_path,caption,clipscore,aesthetic_score
597,src/data/data/laion_aesthetic_subset_big\00171...,Shades Of Gray - stradina a St Paul de Vence b...,0.324271,7.076888
611,src/data/data/laion_aesthetic_subset_big\00175...,Tina Turner Rights Managed Images - stradina a...,0.301922,7.076888
796,src/data/data/laion_aesthetic_subset_big\00223...,Abstract Expressionism - stradina a St Paul de...,0.322866,7.076888
1388,src/data/data/laion_aesthetic_subset_big\00391...,Urban Abstracts - stradina a St Paul de Vence ...,0.332348,7.076888
1653,src/data/data/laion_aesthetic_subset_big\00469...,Just Desserts - stradina a St Paul de Vence by...,0.325831,7.076888
